In [43]:
# 파이썬으로 말뭉치 전처리하기 
# 2.3.1

text = 'You say goodbye and I say hello.'

text = text.lower()  # 소문자 변환
print("text.lower() : ",text)
text = text.replace('.','.')
print("text.replace : ", text)

words = text.split(' ') # 공백을 기준으로 분할
print("text.split : ", words)

text.lower() :  you say goodbye and i say hello.
text.replace :  you say goodbye and i say hello.
text.split :  ['you', 'say', 'goodbye', 'and', 'i', 'say', 'hello.']


In [44]:
# 문장 단어 목록 형태로 이용

word_to_id = {}
id_to_word = {}

for word in words:
    if word not in word_to_id:
        new_id = len(word_to_id)
        word_to_id[word] = new_id
        id_to_word[new_id] = word

In [45]:
print("word_to_id : ",word_to_id) 
print("id_to_word : ",id_to_word)

word_to_id :  {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello.': 5}
id_to_word :  {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello.'}


In [65]:
# 말뭉치 전처리
import numpy as np


def preprocess(text):
    text = text.lower()
    text = text.replace('.', ' .')
    words = text.split(' ')

    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word

    corpus = np.array([word_to_id[w] for w in words])

    return corpus, word_to_id, id_to_word

In [66]:
text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

In [67]:
print("corpus : ",corpus)
print("word_to_id : ",word_to_id)
print("id_to_word : ",id_to_word)

corpus :  [0 1 2 3 4 1 5 6]
word_to_id :  {'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
id_to_word :  {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [68]:
# 2.3.4 동시발생 행렬
# 전처리부터
import sys
sys.path.append('..')
import numpy as np

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print('corpus', corpus)

print('id_to_word ', id_to_word)

# 단어 수 7개 


corpus [0 1 2 3 4 1 5 6]
id_to_word  {0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


In [69]:
# 동시발생 행렬

C = np.array([
    [0,1,0,0,0,0,0],
    [1,0,1,0,1,1,0],
    [0,1,0,1,0,0,0],
    [0,0,1,0,1,0,0],
    [0,1,0,1,0,0,0],
    [0,1,0,0,0,0,1],
    [0,0,0,0,0,1,0], # 동시발생 행렬 구현
], dtype=np.int32)

print('C[0] : ',C[0]) # ID가 0인 단어의 벡터 표현
print('C[4] : ',C[4]) # ID가 4인 단어의 벡터 표현
print('C[word_to_id["goodbye"]] : ', C[word_to_id['goodbye']]) # "goodbye"의 벡터 표현

C[0] :  [0 1 0 0 0 0 0]
C[4] :  [0 1 0 1 0 0 0]
C[word_to_id["goodbye"]] :  [0 1 0 1 0 0 0]


In [70]:
# 말뭉치로부터 동시발생 행렬을 만들어주는 함수
# 자동화

def create_co_matrix(corpus, vocab_size, window_size=1): # 차례로 단어 ID의 리스트, 어휘 수, 윈도우 크기 
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size + 1):
            left_idx = idx - i
            right_idx = idx + i 
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus_size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix

In [71]:
# 코사인 유사도

def cos_similarity(x, y, eps=1e-8):
    mx = x/np.sqrt(np.sum(x**2) + eps) # x의 정규화
    my = y/np.sqrt(np.sum(y**2) + eps) # y의 정규화
    return np.dot(mx, my) # 인수로 제로 벡터(원소가 모두 0인 벡터)가 들어오면 0으로 나누기 오류가 발생
# 해결법은 cos_similarity(x, y, eps=1e-8)로 수정

In [72]:
import sys
sys.path.append('..')
text = "You say goodbye and I say hello."
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

c0 = C[word_to_id['you']] # 'you' 의 단어 벡터
c1 = C[word_to_id['i']] # 'i'의 단어 벡터
print(cos_similarity(c0,c1))


0.7071067758832467


In [73]:
# 유사 단어의 랭킹 표시

def most_similar(query, word_to_id, id_to_word, word_matrix, top=5): # 5개까지
    # 검색어 꺼내기
    if query not in word_to_id:
        print("%s(을)를 찾을 수 없습니다." % query)
        return
    
    print('\n[query]' + query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    # 코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size):
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
        
    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1 * similarity).argsort():
        if id_to_word[i] == query:
            continue
        print(' %s : %s ' %(id_to_word[i], similarity[i]))
        
        count+=1
        if count >= top:
            return

In [74]:
import sys
sys.path.append('..')

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)

most_similar('you', word_to_id, id_to_word, C , top= 5)


[query]you
 goodbye : 0.7071067758832467 
 i : 0.7071067758832467 
 hello : 0.7071067758832467 
 say : 0.0 
 and : 0.0 


In [121]:

def ppmi(C, verbose=False, eps = 1e-8):
    '''PPMI(점별 상호정보량) 생성
    :param C: 동시발생 행렬
    :param verbose: 진행 상황을 출력할지 여부
    :return:
    '''
    M = np.zeros_like(C, dtype=np.float32)
    N = np.sum(C)
    S = np.sum(C, axis=0)
    total = C.shape[0] * C.shape[1]
    cnt = 0

    for i in range(C.shape[0]):
        for j in range(C.shape[1]):
            pmi = np.log2(C[i, j] * N / (S[j]*S[i]) + eps)
            M[i, j] = max(0, pmi)

            if verbose:
                cnt += 1
                if cnt % (total//100) == 0:
                    print('%.1f%% 완료' % (100*cnt/total))
    return M

In [117]:
import sys
sys.path.append('..')

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)

np.set_printoptions(precision=3) # 유효 자릿수를 세 자리로 표시
print('동시발생 행렬 : ', C)
print('-' * 50)
print('PPMI : ', W)

동시발생 행렬 :  [[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]
--------------------------------------------------
PPMI :  [[0.    1.807 0.    0.    0.    0.    0.   ]
 [1.807 0.    0.807 0.    0.807 0.807 0.   ]
 [0.    0.807 0.    1.807 0.    0.    0.   ]
 [0.    0.    1.807 0.    1.807 0.    0.   ]
 [0.    0.807 0.    1.807 0.    0.    0.   ]
 [0.    0.807 0.    0.    0.    0.    2.807]
 [0.    0.    0.    0.    0.    2.807 0.   ]]


In [118]:
# SVD에 의한 차원 감소

import sys
sys.path.append('..')

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)
vocab_size = len(word_to_id)
C = create_co_matrix(corpus, vocab_size)
W = ppmi(C)

# SVD
U, S, V = np.linalg.svd(W)

print("동시발생 행렬 : ",C[0]) # 동시발생 행렬

print("PPMI행렬 : ",W[0]) # PPMI 행렬

print("SVD : ",U[0]) # SVD



동시발생 행렬 :  [0 1 0 0 0 0 0]
PPMI행렬 :  [0.    1.807 0.    0.    0.    0.    0.   ]
SVD :  [-3.409e-01 -1.110e-16 -3.886e-16 -1.205e-01  0.000e+00  9.323e-01
  2.226e-16]


In [119]:
# PTB 데이터셋

import sys
sys.path.append('..')
import pbt

corpus, word_to_id, id_to_word = pbt.load_data('train')

print('말뭉치 크기 : ',len(corpus))
print('corpus[:30] : ', corpus[:30])
print('id_to_word[0] : ', id_to_word[0])
print('id_to_word[1] : ', id_to_word[1])
print('id_to_word[2] : ', id_to_word[2])
print()
print('word_to_id["car"] : ',word_to_id['car'])
print('word_to_id["happy"] : ',word_to_id['happy'])
print('word_to_id["lexus"] : ',word_to_id['lexus'])


말뭉치 크기 :  929589
corpus[:30] :  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29]
id_to_word[0] :  aer
id_to_word[1] :  banknote
id_to_word[2] :  berlitz

word_to_id["car"] :  3856
word_to_id["happy"] :  4428
word_to_id["lexus"] :  7426


In [123]:
# PTB 데이터셋 평가
# PTB 데이터셋

import sys
sys.path.append('..')
import pbt

window_size = 2
wordvec_size = 100

corpus, word_to_id, id_to_word = pbt.load_data('train')
vocab_size = len(word_to_id)
print('동시발생 수 계산 ... ')
C = create_co_matrix(corpus, vocab_size, window_size)
print('PPMI 계산 ...')
W = ppmi(C, verbose=True)

print('SVD 계산 ...')
try:
    # truncated SVD(빠르다.)
    from sklearn.utils.extmath import randomized_svd
    U, S, V = randomized_svd(W, n_components=wordvec_size, n_iter=5, random_state=None)
    
except ImportError:
    # SVD(느리다.)
    U, S, V = np.linalg.svd(W)
    
word_vecs = U[:, :wordvec_size]

querys = ['you','year','car','toyota']

for query in querys:
    most_similar(query, word_to_id, id_to_word, word_vecs, top=5)

동시발생 수 계산 ... 
PPMI 계산 ...


C:\Users\as_th\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: overflow encountered in long_scalars
  from ipykernel import kernelapp as app
C:\Users\as_th\anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in log2
  from ipykernel import kernelapp as app


1.0% 완료
2.0% 완료
3.0% 완료
4.0% 완료
5.0% 완료
6.0% 완료
7.0% 완료
8.0% 완료
9.0% 완료
10.0% 완료
11.0% 완료
12.0% 완료
13.0% 완료
14.0% 완료
15.0% 완료
16.0% 완료
17.0% 완료
18.0% 완료
19.0% 완료
20.0% 완료
21.0% 완료
22.0% 완료
23.0% 완료
24.0% 완료
25.0% 완료
26.0% 완료
27.0% 완료
28.0% 완료
29.0% 완료
30.0% 완료
31.0% 완료
32.0% 완료
33.0% 완료
34.0% 완료
35.0% 완료
36.0% 완료
37.0% 완료
38.0% 완료
39.0% 완료
40.0% 완료
41.0% 완료
42.0% 완료
43.0% 완료
44.0% 완료
45.0% 완료
46.0% 완료
47.0% 완료
48.0% 완료
49.0% 완료
50.0% 완료
51.0% 완료
52.0% 완료
53.0% 완료
54.0% 완료
55.0% 완료
56.0% 완료
57.0% 완료
58.0% 완료
59.0% 완료
60.0% 완료
61.0% 완료
62.0% 완료
63.0% 완료
64.0% 완료
65.0% 완료
66.0% 완료
67.0% 완료
68.0% 완료
69.0% 완료
70.0% 완료
71.0% 완료
72.0% 완료
73.0% 완료
74.0% 완료
75.0% 완료
76.0% 완료
77.0% 완료
78.0% 완료
79.0% 완료
80.0% 완료
81.0% 완료
82.0% 완료
83.0% 완료
84.0% 완료
85.0% 완료
86.0% 완료
87.0% 완료
88.0% 완료
89.0% 완료
90.0% 완료
91.0% 완료
92.0% 완료
93.0% 완료
94.0% 완료
95.0% 완료
96.0% 완료
97.0% 완료
98.0% 완료
99.0% 완료
100.0% 완료
SVD 계산 ...

[query]you
 we : 0.6305503845214844 
 i : 0.6273680925369263 
 do : 0.5275903940200806 
 anybody